In [ ]:
%pwd

In [ ]:
import numpy as np
#import Atom
import Molecule
#import Ring
import OPLS as op
import System
import Conjugated_Polymer
import Cluster_IO
#import Write_Inputs
#import Write_Submit_Script
import math
import copy
#import scipy
import os
import sys
import matplotlib.pyplot as plt
#import matplotlib.cm
import matplotlib as mpl
from symfit import parameters, variables, sin, cos, Fit
#import re
#import time
#import dill
from aromodel_lib import *

In [ ]:
#File_Location = "~/Torsional_Parameterization"
#os.chdir("/Users/andrewkleinschmidt/Test_New_Torsional_Setup")
Input_File = "./test/P3HT_Input.txt"
XYZ_File = "./test/P3HT_Input.xyz"
#Input_File_Sidechains = "P3HT_Input_With_Sidechains.txt"
#XYZ_File_Sidechains = "P3HT_Input_With_Sidechains.xyz"
Input_File_Sidechains = Input_File
XYZ_File_Sidechains = XYZ_File
Polymer_Name = "P3HT"
Rotated_Shape = (7,36)
Min_Dih = 0
Max_Dih = 350
Min_OOP = 0
Max_OOP = 60
Fine_Rotated_Shape = (7,36)
Fine_Min_Dih = 0
Fine_Max_Dih = 360
Fine_Min_OOP = 0
Fine_Max_OOP = 30

OOP_Rotations = np.linspace(Min_OOP,Max_OOP,Rotated_Shape[0])
Dih_Rotations = np.linspace(Min_Dih,Max_Dih,Rotated_Shape[1])

In [ ]:
%pwd

In [ ]:
def Set_Up_Folders(path):
    folders = ["Bond_Parameters","Figures","Rotation_Run_Input_Copies","XYZ_Files","Hydrogenated_XYZ_Files",
    "Alternate_Hydrogenated_XYZ_Files","Nontorsional_Outputs","Nontorsional_Inputs","Hydrogenated_Improper_XYZ_Files",
    "Alternate_Hydrogenated_Improper_XYZ_Files","Multi_Ring_Hydrogenated_Rotation_Test", ]
    for folder in folders:
        comb_path = path+"/"+folder
        if (not os.path.exists(path+"/"+folder)):
            os.mkdir(comb_path)

In [ ]:
Set_Up_Folders('.')

In [ ]:
#This function reads in the parameter file, assigns the atoms to rings, adds available LJ, coulombic, bonded, angular, dihedral, and improper potentials, 
#and tells the program whether it needs to parameterize missing bond potentials or partial charges. 
#Ring_List: NumPy array of Ring objects categorizing all available atoms into separate rings; 
#Paramaterize_Bond: Boolean that equals "True" if bond parameters for interring bonds have not been specified; 
#Paramaterize_Charges: Boolean that equals "True" if partial charges for atoms have not been specified

Ring_List,Parameterize_Bond,Parameterize_Charges = Read_Input(Input_File,XYZ_File,Polymer_Name) 
# cpu/0.15.4  gcc/9.2.0  openmpi/3.1.6

In [ ]:
%pwd
print(len(Ring_List))

In [ ]:
if Parameterize_Charges:
    Find_Charges(Ring_List,Polymer_Name) #Assign charges to each atom in each ring

In [ ]:
if Parameterize_Bond:
    Strech_Bond(Ring_List,Polymer_Name) #Assign spring constants and equilibrium lengths to each interring potential

In [ ]:
Make_Example_Files(Ring_List)

In [ ]:
Offset_Ring_List = []
for ring in Ring_List[1:]:
    Offset_Ring_List.append(copy.deepcopy(ring))

In [ ]:
Offset_Ring_List.append(copy.deepcopy(Ring_List[0]))
for ring1,ring2 in zip(Ring_List,Offset_Ring_List):
    Dimer = Conjugated_Polymer.Conjugated_Polymer([ring1,ring2])
    XYZ_Filename = Dimer.Write_XYZ()
    Hydrogenated_Dimer,_ = Dimer.Create_Hydrogenated_Copy(0,1)
    print(Hydrogenated_Dimer.Ring_List[0].Name)
    print(Hydrogenated_Dimer.Ring_List[0].Normal_Vector)
    print(Hydrogenated_Dimer.Ring_List[1].Normal_Vector)
    Hydrogenated_Dimer.Ring_List[1].Show_Normal_Vector()
    XYZ_Filename = Hydrogenated_Dimer.Write_XYZ()

In [ ]:
Offset_Ring_List = []
Dual_Offset_Ring_List = []
Full_Trimer_Nontorsional_Energy_List = []
for ring in Ring_List[1:]:
    Offset_Ring_List.append(copy.deepcopy(ring))
for ring in Ring_List[2:]:
    Dual_Offset_Ring_List.append(copy.deepcopy(ring))

Offset_Ring_List.append(copy.deepcopy(Ring_List[0]))
Dual_Offset_Ring_List.append(copy.deepcopy(Ring_List[0]))
Dual_Offset_Ring_List.append(copy.deepcopy(Ring_List[1]))

In [ ]:
for ring1,ring2,ring3 in zip(Ring_List,Offset_Ring_List,Dual_Offset_Ring_List):
    Trimer = Conjugated_Polymer.Conjugated_Polymer([ring1,ring2,ring3])
    XYZ_Filename = Trimer.Write_XYZ()

In [ ]:
Extended_Ring_List = copy.deepcopy(Ring_List)
for i in range(1):
    for j in range(1):
        Extended_Ring_List.append(copy.deepcopy(Ring_List[j]))
Multimer = Conjugated_Polymer.Conjugated_Polymer(Extended_Ring_List)
Multimer.Write_XYZ()

In [ ]:
#Forms n-mers of the rings up until a percentage limit of change is reached, then rotates their rings and calculates RI-MP2 energies, LAMMPS energies without bonded interactions, and nonbonded interaction energies; Returns N: the maximum number of rings tested; Torsional_Energies: an N by len(Ring_List) by Rotated_Shape[0] by Rotated_Shape[1]
LAMMPS_Energies = np.zeros(Rotated_Shape)
Quantum_Energies = np.zeros(Rotated_Shape)
Nonbonded_Energies = np.zeros(Rotated_Shape)

Offset_Ring_List = []
Dual_Offset_Ring_List = []
for ring in Ring_List[1:]:
    Offset_Ring_List.append(ring)

for ring in Ring_List[2:]:
    Dual_Offset_Ring_List.append(ring)

Offset_Ring_List.append(Ring_List[0])
Dual_Offset_Ring_List.append(Ring_List[0])
Dual_Offset_Ring_List.append(Ring_List[1])

Dih_Rotations_Degrees = np.linspace(0,Max_Dih + Max_Dih/Rotated_Shape[1]-1,Rotated_Shape[1]+1)
OOP_Rotations_Degrees = np.linspace(0,Max_OOP + Max_OOP/Rotated_Shape[0]-1,Rotated_Shape[0]+1)

In [ ]:
#Calculate_Statistics(Input_File_Sidechains,XYZ_File_Sidechains,Polymer_Name,15,3)

In [ ]:
Run_SPE_Methyl_Impropers(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
Run_SPE_Methyl_Impropers(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_Dual_Hydrogenated_End_File_Matrices = Run_Paired_Hydrogenation_Energy(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices,Ring_By_Ring_Nontorsional_Energy,Ring_By_Ring_Improper_File_Matrices = Run_SPE_Dimers(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)


In [ ]:
Ring_By_Ring_End_File_Matrices_Fine,Ring_By_Ring_Nontorsional_Energy_Fine,Ring_By_Ring_Improper_File_Matrices_Fine = Run_SPE_Dimers(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated,Ring_By_Ring_Hydrogenated_Energy = Run_SPE_Dimers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Fine,Ring_By_Ring_Hydrogenated_Energy_Fine = Run_SPE_Dimers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Alternate,Ring_By_Ring_Hydrogenated_Energy_Alternate = Run_SPE_Dimers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Alternate=True)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Alternate_Fine,Ring_By_Ring_Hydrogenated_Energy_Alternate_Fine = Run_SPE_Dimers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Alternate=True)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers,Ring_By_Ring_Nontorsional_Energy_HI = Run_SPE_Impropers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Fine,Ring_By_Ring_Nontorsional_Energy_HI_Fine = Run_SPE_Impropers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Alternate,Ring_By_Ring_Nontorsional_Energy_HI_Alternate = Run_SPE_Impropers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Alternate=True)

In [ ]:
Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Alternate_Fine,Ring_By_Ring_Nontorsional_Energy_HI_Alternate_Fine = Run_SPE_Impropers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Alternate=True)

In [ ]:
#Ring_By_Ring_End_File_Matrices_Trimer,Ring_By_Ring_Trimer_Nontorsional_Energy = Run_SPE_Trimers_Dih(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
#Ring_By_Ring_End_File_Matrices_Hydrogenated_Trimer,Syn_Anti_Matrices = Run_SPE_Trimers_Hydrogenated_Dih(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name)

In [ ]:
import imp
import aromodel_lib
imp.reload(aromodel_lib)
imp.reload(Cluster_IO)
from aromodel_lib import *

In [ ]:
Ring_By_Ring_Methyl_Impropers,Ring_By_Ring_Methyl_Improper_Lists,Dimer_Names = Return_SPE_Methyl_Impropers(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_Improper_File_Matrices)

In [ ]:
Ring_By_Ring_Methyl_Impropers_Fine,Ring_By_Ring_Methyl_Improper_Lists_Fine,Dimer_Names_Fine = Return_SPE_Methyl_Impropers(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_Improper_File_Matrices_Fine)

In [ ]:
Ring_By_Ring_Hydrogenated_Energies = Return_SPE_Dimers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated)

In [ ]:
Ring_By_Ring_Hydrogenated_Energies_Fine = Return_SPE_Dimers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Fine)

In [ ]:
Ring_By_Ring_Hydrogenated_Alternate_Energies = Return_SPE_Dimers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Alternate,Alternate=True)

In [ ]:
Ring_By_Ring_Hydrogenated_Alternate_Energies_Fine = Return_SPE_Dimers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Alternate_Fine,Alternate=True)

In [ ]:
Combined_Ring_By_Ring_Hydrogenated_Energies,Combined_Ring_By_Ring_Hydrogenated_Energies_Tracker = Merge_Hydrogenated_Energies(Ring_By_Ring_Hydrogenated_Energies,Ring_By_Ring_Hydrogenated_Alternate_Energies)

In [ ]:
Combined_Ring_By_Ring_Hydrogenated_Energies_Fine,Combined_Ring_By_Ring_Hydrogenated_Energies_Tracker_Fine = Merge_Hydrogenated_Energies(Ring_By_Ring_Hydrogenated_Energies_Fine,Ring_By_Ring_Hydrogenated_Alternate_Energies_Fine)

In [ ]:
Ring_By_Ring_Hydrogenated_Improper_Energies = Return_SPE_Dimers_Impropers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers)

In [ ]:
Ring_By_Ring_Hydrogenated_Improper_Energies_Fine = Return_SPE_Dimers_Impropers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Fine)

In [ ]:
Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies = Return_SPE_Dimers_Impropers_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Alternate,Alternate=True)

In [ ]:
Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies_Fine = Return_SPE_Dimers_Impropers_Hydrogenated(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Impropers_Alternate_Fine,Alternate=True)

In [ ]:
Combined_Ring_By_Ring_Hydrogenated_Improper_Energies,Combined_Ring_By_Ring_Hydrogenated_Improper_Energies_Tracker = Merge_Hydrogenated_Improper_Energies(Ring_By_Ring_Hydrogenated_Improper_Energies,Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies,Combined_Ring_By_Ring_Hydrogenated_Energies_Tracker)

In [ ]:
Combined_Ring_By_Ring_Hydrogenated_Improper_Energies_Fine,Combined_Ring_By_Ring_Hydrogenated_Improper_Energies_Tracker_Fine = Merge_Hydrogenated_Improper_Energies(Ring_By_Ring_Hydrogenated_Improper_Energies_Fine,Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies_Fine,Combined_Ring_By_Ring_Hydrogenated_Energies_Tracker_Fine)

In [ ]:
Dimer_Energies,Dimer_Energies_Dict,Dimer_Energies_Corrected = Return_SPE_Dimers(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices,Ring_By_Ring_Nontorsional_Energy,Ring_By_Ring_Hydrogenated_Energies)

In [ ]:
Dimer_Energies_Fine,Dimer_Energies_Dict_Fine,Dimer_Energies_Corrected_Fine = Return_SPE_Dimers(Ring_List,Fine_Rotated_Shape,Fine_Max_Dih,Fine_Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Fine,Ring_By_Ring_Nontorsional_Energy_Fine,Ring_By_Ring_Hydrogenated_Energies_Fine)

In [ ]:
OPLS_Fit_list = Control_Fit_OPLS(Dimer_Energies,Rotated_Shape)

In [ ]:
#Merged_OOP_Rotations_Degrees,Merged_Raw_Dimer_Energies = Merge_Coarse_Fine(Rotated_Shape,Max_OOP,Max_Dih,Fine_Rotated_Shape,Fine_Max_OOP,Fine_Max_Dih,Dimer_Energies,Dimer_Energies_Fine)

In [ ]:
#Check this-- Merged Raw Dimer Energies is based on corrected #. Doesn't appear to matter but still
Merged_OOP_Rotations_Degrees,Merged_Raw_Dimer_Energies = Merge_Coarse_Fine(Rotated_Shape,Max_OOP,Max_Dih,Fine_Rotated_Shape,Fine_Max_OOP,Fine_Max_Dih,Dimer_Energies_Corrected,Dimer_Energies_Corrected_Fine)

In [ ]:
Dimer_Weights = Weight_Impropers(Merged_Raw_Dimer_Energies,5000)

In [ ]:
#Dually_Hydrogenated_Energies = Return_Dually_Hydrogenated(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_Dual_Hydrogenated_End_File_Matrices,Syn_Anti_Matrices)

In [ ]:
Ring_By_Ring_Nonbonded_Energies = np.asarray(Combined_Ring_By_Ring_Hydrogenated_Energies) - np.asarray(Combined_Ring_By_Ring_Hydrogenated_Improper_Energies)

In [ ]:
Nonbonded_Energies = np.array(Ring_By_Ring_Hydrogenated_Energies) - np.array(Ring_By_Ring_Hydrogenated_Improper_Energies)
Nonbonded_Energies_Alternate = np.array(Ring_By_Ring_Hydrogenated_Alternate_Energies) - np.array(Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies)

In [ ]:
Ring_By_Ring_Nonbonded_Energies,_ = Merge_Hydrogenated_Energies(Nonbonded_Energies,Nonbonded_Energies_Alternate)

In [ ]:
for names,energies in zip(Dimer_Names,Ring_By_Ring_Nonbonded_Energies):
    energies = energies - np.amin(energies[0])
    fig,ax = plt.subplots(1,1)
    x,y = np.meshgrid(Dih_Rotations_Degrees,OOP_Rotations_Degrees)
    c = ax.pcolor(x,y,energies,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Nonbonded Energies',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Nonbonded_Energies' % (names[0],names[1]))
    plt.close(fig)

for ring in Ring_By_Ring_Nonbonded_Energies:
    ring = ring - np.amin(ring[0])

In [ ]:
E_conjugation = np.asarray(Ring_By_Ring_Methyl_Impropers)
Dimer_Energies = np.asarray(Dimer_Energies)

E_delocalization = Dimer_Energies - E_conjugation - Ring_By_Ring_Nonbonded_Energies
#E_delocalization = Dimer_Energies - Ring_By_Ring_Nonbonded_Energies

In [ ]:
Dih_Rotations_Degrees = np.linspace(0,Max_Dih + Max_Dih/Rotated_Shape[1]-1,Rotated_Shape[1]+1)
OOP_Rotations_Degrees = np.linspace(0,Max_OOP + Max_OOP/Rotated_Shape[0]-1,Rotated_Shape[0]+1)
print(E_delocalization.shape)
print(len(Dih_Rotations_Degrees))
print(len(OOP_Rotations_Degrees))
Make_Surface_Plot(Dih_Rotations_Degrees,OOP_Rotations_Degrees,E_delocalization[0],'%s_Unfit_Conjugated_Energies' % (Polymer_Name),Title='RI-MP2 Energies',xlabel='Dihedral (degrees)',ylabel='Out of Plane (degrees)')

In [ ]:
Ring_By_Ring_Nonbonded_Energies_Fine = np.asarray(Combined_Ring_By_Ring_Hydrogenated_Energies_Fine) - np.asarray(Combined_Ring_By_Ring_Hydrogenated_Improper_Energies_Fine)

Nonbonded_Energies_Fine = np.array(Ring_By_Ring_Hydrogenated_Energies_Fine) - np.array(Ring_By_Ring_Hydrogenated_Improper_Energies_Fine)
Nonbonded_Energies_Alternate_Fine = np.array(Ring_By_Ring_Hydrogenated_Alternate_Energies_Fine) - np.array(Ring_By_Ring_Hydrogenated_Alternate_Improper_Energies_Fine)
Ring_By_Ring_Nonbonded_Energies_Fine,_ = Merge_Hydrogenated_Energies(Nonbonded_Energies_Fine,Nonbonded_Energies_Alternate_Fine)

for ring in Ring_By_Ring_Nonbonded_Energies:
    ring = ring - np.amin(ring[0])

E_conjugation_Fine = np.asarray(Ring_By_Ring_Methyl_Impropers_Fine)
Dimer_Energies_Fine = np.asarray(Dimer_Energies_Fine)
E_delocalization_Fine = Dimer_Energies_Fine - E_conjugation_Fine - Ring_By_Ring_Nonbonded_Energies_Fine
#E_delocalization_Fine = Dimer_Energies_Fine - Ring_By_Ring_Nonbonded_Energies_Fine

In [ ]:
Merged_OOP_Rotations_Degrees,Merged_Delocalization_Energies = Merge_Coarse_Fine(Rotated_Shape,Max_OOP,Max_Dih,Fine_Rotated_Shape,Fine_Max_OOP,Fine_Max_Dih,E_delocalization,E_delocalization_Fine)

In [ ]:
Merged_OOP_Rotations_Degrees,Merged_Conjugation_Energies = Merge_Coarse_Fine(Rotated_Shape,Max_OOP,Max_Dih,Fine_Rotated_Shape,Fine_Max_OOP,Fine_Max_Dih,E_conjugation,E_conjugation_Fine)

In [ ]:
Merged_OOP_Rotations_Degrees,Merged_Nonbonded_Energies = Merge_Coarse_Fine(Rotated_Shape,Max_OOP,Max_Dih,Fine_Rotated_Shape,Fine_Max_OOP,Fine_Max_Dih,Ring_By_Ring_Nonbonded_Energies,Ring_By_Ring_Nonbonded_Energies_Fine)

In [ ]:
import imp
import aromodel_lib
imp.reload(aromodel_lib)
from aromodel_lib import *
Force_x,Force_y,Fit_Energies,Deloc_Coarse_Energies,a_params_list,b_params_list,a0_params_list = Fit_Improper_Energies(Merged_Delocalization_Energies,Ring_List,Rotated_Shape,Merged_OOP_Rotations_Degrees,Dimer_Weights)

In [ ]:
Nonbonded_Force_x,Nonbonded_Force_y,Nonbonded_Fit_Energies,Nonbonded_Coarse_Energies,Nonbonded_a_params_list,Nonbonded_b_params_list,Nonbonded_a0_params_list = Fit_Improper_Energies(Merged_Nonbonded_Energies,Ring_List,Rotated_Shape,Merged_OOP_Rotations_Degrees,Dimer_Weights,Nonbonded=True)

In [ ]:
Conjugated_Parameters = Fit_Conjugated_Energies(Merged_Conjugation_Energies,Merged_OOP_Rotations_Degrees)

In [ ]:
#Write_Example_Plumed_Script(Polymer_Name,"N2200_Input.txt","N2200_Input.xyz",Fit_Energies,Force_y,Force_x,1,Folder_Name = "Dimer_Plumed_PNDI_T") #Plumed Script for dimer w/o sidechains

In [ ]:
#dill.dump_session('N2200_env.db')
import imp
import aromodel_lib
import Molecule
imp.reload(System)
imp.reload(aromodel_lib)
imp.reload(Molecule)
from aromodel_lib import *
print(Ring_List,Ring_List[1].Dihedral_List)
Write_All_Dimers_Plumed(Ring_List,Fit_Energies,Force_y,Force_x,Polymer_Name)

In [ ]:
print(Polymer_Name)#TODO: Figure out why I need to manually add _Input to the end
Calculate_Conventional_Energies(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name + "_Input",OPLS_Fit_list)

In [ ]:
#os.chdir("/Users/andrewkleinschmidt/Test_New_Torsional_Setup")
#dill.load_session("N2200_env.db")
#TODO: there is way too much hard code in this function, like wtf
Check_New_Torsional_Energy(Ring_List,Polymer_Name+"_Input",Rotated_Shape,a_params_list,b_params_list,a0_params_list,OPLS_Fit_list,Conjugated_Parameters,Nonbonded_a_params_list,Nonbonded_b_params_list,Nonbonded_a0_params_list)

In [ ]:
Compare_Rigidity_Types(Merged_Conjugation_Energies,Merged_Delocalization_Energies,Merged_Nonbonded_Energies,Merged_OOP_Rotations_Degrees)

In [ ]:
imp.reload(aromodel_lib)
from aromodel_lib import *
Write_Conventional_Data_File(Polymer_Name,Input_File_Sidechains,XYZ_File_Sidechains,15,OPLS_Fit_list,Folder_Name="Conventional_Scripts")

In [ ]:
imp.reload(aromodel_lib)
from aromodel_lib import *
Write_Conventional_Data_File(Polymer_Name + "_Noninteracting",Input_File_Sidechains,XYZ_File_Sidechains,15,OPLS_Fit_list,Folder_Name="Conventional_Scripts",Non_Interacting=True)

In [ ]:
Conjugated_Ring_List = []
for i in range(0,len(Ring_List),2):
    Conjugated_Ring_List.append(Ring_List[i+1])
    Conjugated_Ring_List.append(Ring_List[i])

for ring,params in zip(Conjugated_Ring_List,Conjugated_Parameters):
    for b_atom in ring.Bonded_Atoms:
        b_atom.Add_Improper_Params(params[0],params[1],params[2])

In [ ]:
Polymer_Rings,Parameterize_Bond,Parameterize_Charges = Read_Input(Input_File_Sidechains,XYZ_File_Sidechains,Polymer_Name)

Conjugated_Polymer_Ring_List = []
for i in range(0,len(Polymer_Rings),2):
    Conjugated_Polymer_Ring_List.append(Polymer_Rings[i+1])
    Conjugated_Polymer_Ring_List.append(Polymer_Rings[i])

for ring,params in zip(Conjugated_Polymer_Ring_List,Conjugated_Parameters):
    for b_atom in ring.Bonded_Atoms:
        b_atom.Add_Improper_Params(params[0],params[1],params[2])

In [ ]:
Polymer_Ring_List = []
for i in range(15):
    for ring in Polymer_Rings:
        Polymer_Ring_List.append(copy.deepcopy(ring))

for ring,params in zip(Polymer_Ring_List,Conjugated_Parameters):
    for b_atom in ring.Bonded_Atoms:
        b_atom.Add_Improper_Params(params[0],params[1],params[2])

In [ ]:
imp.reload(aromodel_lib)
from aromodel_lib import *
Write_Example_Plumed_Script(Polymer_Ring_List,Fit_Energies,Force_y,Force_x,Polymer_Name,Folder_Name="./Full_Polymer_Plumed") #Plumed Script for full polymer
Write_Example_Plumed_Script(Polymer_Ring_List,Fit_Energies,Force_y,Force_x,Polymer_Name+"_Noninteracting",Folder_Name="./Full_Polymer_Plumed",Non_Interacting=True,Nonbonded_Fit_Energies=Nonbonded_Fit_Energies,Nonbonded_Force_x=Nonbonded_Force_x,Nonbonded_Force_y=Nonbonded_Force_y) #Plumed Script for full polymer with no LJ and Coulombic interactions

In [ ]:
imp.reload(Molecule)
imp.reload(Conjugated_Polymer)
imp.reload(System)
imp.reload(Ring)
import sys
sys.setrecursionlimit(10000)
Polymer = Conjugated_Polymer.Conjugated_Polymer(Polymer_Ring_List)

Bond_Atoms = []
for bond in Polymer.Interring_Bond_List:
        if bond.Bond_Main not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Main)
        if bond.Bond_Node not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Node)
System_List = []
#Comp_List = np.ones(60,dtype=int)
Comp_List = np.ones(30,dtype=int)
#for poly in range(60):
for poly in range(30):
    Polymer = Conjugated_Polymer.Conjugated_Polymer(Polymer_Ring_List)
    #Polymer.Replace_Interring_Dihedrals([0],[0],[0.0,0.0,0.0,0.0]) Changed by Leon
    Polymer.Replace_Interring_Dihedrals([0],[0],[[0.0,0.0,0.0,0.0]])
    Molecule.Assign_Lammps([Polymer])
    for atom in Polymer.Atom_List:
        atom.Charge = 0.0
        atom.Epsilon = 0.0
        atom.Sigma = 4.0
    Polymer.Randomize_Torsions(Deloc_Coarse_Energies[0][3:-3]/4.184,Nonbonded_Coarse_Energies[0][3:-3]/4.184)
    System_List.append(copy.deepcopy(Polymer))
#Film = System.System(System_List,Comp_List,80.0,"PNDI_T_Film_60_30mers")
Film = System.System(System_List,Comp_List,64.0,"P3HT_Film_30_30mers_COLVAR_torsion")
Film.Gen_Rand()
Film.Write_LAMMPS_Data()
Film.Write_Plumed_Files(Comp_List,"P3HT_Film",Fit_Energies,Force_y,Force_x,Reverse = True,Folder_Name="./Full_Polymer_Plumed")
#Polymer.Write_Data_File("Test_Randomize.data",Exclude_Interring_Dihedrals=True,Bond_Atoms=Bond_Atoms,Soft_Potential=True)

In [ ]:
import sys
sys.setrecursionlimit(10000)
Conventional_Polymer = Conjugated_Polymer.Conjugated_Polymer(Polymer_Ring_List)
Molecule.Assign_Lammps([Conventional_Polymer])
Conventional_Polymer.Replace_Interring_Dihedrals(0,0,OPLS_Fit_list[0])
Bond_Atoms = []
for bond in Conventional_Polymer.Interring_Bond_List:
        if bond.Bond_Main not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Main)
        if bond.Bond_Node not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Node)
System_List = []
#Comp_List = np.ones(60,dtype=int)
Comp_List = np.ones(30,dtype=int)
#for poly in range(60):
for atom in Conventional_Polymer.Atom_List:
    atom.Charge = 0.0
    atom.Epsilon = 0.0
    atom.Sigma = 4.0
for poly in range(30):
    System_List.append(copy.deepcopy(Conventional_Polymer))

#Film = System.System(System_List,Comp_List,80.0,"PNDI_T_Film_60_30mers")
Film = System.System(System_List,Comp_List,64.0,"P3HT_Film_30_30mers_Dihedral_torsion")
Film.Gen_Rand()
Film.Update_Atom_Positions("P3HT_Film_30_30mers_COLVAR_torsion.data")
Film.Write_LAMMPS_Data()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.plot(Dih_Rotations,E_delocalization[0][0],label="Colvar-Torsion")
ax.plot(Dih_Rotations,Dimer_Energies[0][0],label="Dihedral-Torsion")
plt.xlabel("Degrees ()",size=24)
plt.ylabel("Energy (kcal/mol)",size=24)
plt.tight_layout()
ax.legend()
fig.savefig('%s_Figure2' % ("P3HT"))
print()

In [ ]:
Reverse = False
Base_Calculate_Torsions_String = Polymer_Name + "_Calculate_Torsions_Conventional"
index = 1
Num_Atoms = 1
Num_Rings = 0
oop_blocks = np.linspace(0,2*math.pi,200)
dih_blocks = np.linspace(-math.pi,math.pi,200)
torsion_file = open(Base_Calculate_Torsions_String + ".dat", 'w')
for mol in Film.Molecule_List:
    Bond_Atoms = []
    for bond in mol.Interring_Bond_List:
        if bond.Bond_Main not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Main)
        if bond.Bond_Node not in Bond_Atoms:
            Bond_Atoms.append(bond.Bond_Node)

    torsion_file.write("\n\nc%d: CENTER ATOMS=%d" % (index,mol.Ring_List[0].Core_Atom_List[0].System_ID))
    for atom in mol.Ring_List[0].Core_Atom_List[1:]:
        torsion_file.write(",%d" % atom.System_ID)
    torsion_file.write("\n\nc%d_normal: GHOST ATOMS=c1,%d,%d COORDINATES=0.0,1.0,0.0\n\n" % (index,mol.Ring_List[0].Core_Atom_List[0].System_ID,mol.Ring_List[0].Core_Atom_List[1].System_ID))

    for i in range(1,len(mol.Ring_List)):
        index += 1
        for b_atom in mol.Ring_List[i].Bonded_Atoms:
            if b_atom.Is_Linked and b_atom.Interring_Bond_Atom.Self_Ring.Ring_ID == i:
                atom1 = b_atom.Interring_Bond_Atom.Central_Atom.System_ID
                atom2 = b_atom.Central_Atom.System_ID
                ring1_batom1 = b_atom.Interring_Bond_Atom.Same_Ring_Bonded_Atom_List[0].System_ID
                ring1_batom2 = b_atom.Interring_Bond_Atom.Same_Ring_Bonded_Atom_List[1].System_ID
                ring2_batom1 = b_atom.Same_Ring_Bonded_Atom_List[0].System_ID
                ring2_batom2 = b_atom.Same_Ring_Bonded_Atom_List[1].System_ID
        torsion_file.write("\n\nc%d: CENTER ATOMS=%d" % (index,mol.Ring_List[i].Core_Atom_List[0].System_ID))
        for atom in mol.Ring_List[i].Core_Atom_List[1:]:
            torsion_file.write(",%d" % atom.System_ID)
        torsion_file.write("\n\nc%d_normal: GHOST ATOMS=c%d,%d,%d COORDINATES=0.0,1.0,0.0\n\n" % (index,index,mol.Ring_List[i].Core_Atom_List[0].System_ID,mol.Ring_List[i].Core_Atom_List[1].System_ID))
        if Reverse:
            torsion_file.write("DIH_%d: TORSION VECTOR1=c%d_normal,c%d AXIS=%d,%d VECTOR2=c%d_normal,c%d\n\n" % (index-1,index-1,index-1,atom2,atom1,index,index))
        else:
            torsion_file.write("DIH_%d: TORSION VECTOR1=c%d,c%d_normal AXIS=%d,%d VECTOR2=c%d,c%d_normal\n\n" % (index-1,index-1,index-1,atom1,atom2,index,index))
        torsion_file.write("OOP_%d_1: TORSION ATOMS=%d,%d,%d,%d\n\n" % (index-1,atom1,ring1_batom1,ring1_batom2,atom2))
        torsion_file.write("OOP_%d_2: TORSION ATOMS=%d,%d,%d,%d\n\n" % (index-1,atom2,ring2_batom1,ring2_batom2,atom1))
        torsion_file.write("OOP_%d: MATHEVAL ARG=OOP_%d_1,OOP_%d_2 FUNC=abs(x)+abs(y) PERIODIC=NO\n\n" % (index-1,index-1,index-1))
        torsion_file.write("PRINT ARG=DIH_%d,OOP_%d FILE=colvar_%d.txt STRIDE=100\n\n" % (index-1,index-1,index-1))

    torsion_file.write("WHOLEMOLECULES ENTITY0=%d-%d" % (Num_Atoms,Num_Atoms+len(mol.Atom_List)-1))
    Num_Atoms += len(mol.Atom_List)
    Num_Rings += len(mol.Ring_List)
    for i in range(1,len(mol.Ring_List)):
        torsion_file.write(",c%d,c%d_normal" % (i,i))
    index += 1

torsion_file.close()

In [ ]:
Sub_Directory = "./Polymer_Plumed_P3HT"
Colvars = f.readlines()
torsion_index = 1
torsion_indices = []
for polymer_num in range(30):
    for torsion_num in range(30):
        torsion_indices.append(torsion_index)
        torsion_index += 1
    torsion_index += 1
for colvar_num in torsion_indices:
    f = open(Sub_Directory + "/colvar_%d.txt" % colvar_num,'r')
    for line in Colvars[-1::-1]:
        if len(line.strip().split()) == 3 and all((ch.isdigit() or ch == ".") for ch in line.strip().split()[0].strip()):
            Colvar_DIH_List.append(float(line.strip().split()[1].strip()))
            Colvar_OOP_List.append(float(line.strip().split()[2].strip()))
            break
    f.close()

In [ ]:
Nonbonded_Modifiers = [0.0,0.5,1.0,2.0]
Delocalization_Modifiers = [0.0,0.5,1.0,2.0]

for n_mod in Nonbonded_Modifiers:
    for d_mod in Delocalization_Modifiers:
        Write_Example_Plumed_Script(Polymer_Ring_List,Fit_Energies,Force_y,Force_x,Polymer_Name+"_Noninteracting",Folder_Name="./Full_Polymer_Plumed",Non_Interacting=True,Nonbonded_Fit_Energies=Nonbonded_Fit_Energies,Nonbonded_Force_x=Nonbonded_Force_x,Nonbonded_Force_y=Nonbonded_Force_y,Delocalization_Modifier=d_mod,Nonbonded_Modifier=n_mod)

In [ ]:
Make_Solvated_Box(Polymer_Ring_List,Polymer_Name,"Chlorobenzene")

In [ ]:
Ring_By_Ring_Total_Trimer_Energy = Return_SPE_Trimers_Dih(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Trimer,Dimer_Energies_Dict,Ring_By_Ring_Trimer_Nontorsional_Energy)

Ring_By_Ring_Hydrogenated_Trimer_Energy =  Return_SPE_Trimers_Hydrogenated_Dih(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,Ring_By_Ring_End_File_Matrices_Hydrogenated_Trimer,Dimer_Energies_Dict,Ring_By_Ring_Trimer_Nontorsional_Energy)

Calculate_Trimer_Long_Range_Energy(Ring_List,Rotated_Shape,Max_Dih,Max_OOP,Polymer_Name,E_delocalization,Ring_By_Ring_Hydrogenated_Improper_Energies,Ring_By_Ring_Total_Trimer_Energy,Ring_By_Ring_Hydrogenated_Trimer_Energy,Ring_By_Ring_End_File_Matrices_Trimer,Dually_Hydrogenated_Energies,Ring_By_Ring_Nonbonded_Energies)
for names,energies in zip(Dimer_Names,Ring_By_Ring_Nonbonded_Energies):
    energies = energies - np.amin(energies[0])
    fig,ax = plt.subplots(1,1)
    x,y = np.meshgrid(Dih_Rotations_Degrees,OOP_Rotations_Degrees)
    c = ax.pcolor(x,y,energies,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Nonbonded Energies',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Nonbonded_Energies' % (names[0],names[1]))
    plt.close(fig)

for names,reg_tracker,improper_tracker in zip(Dimer_Names,Combined_Ring_By_Ring_Hydrogenated_Energies_Tracker,Combined_Ring_By_Ring_Hydrogenated_Improper_Energies_Tracker):
    energies = energies - np.amin(energies[0])
    fig,ax = plt.subplots(1,1)
    x,y = np.meshgrid(Dih_Rotations_Degrees,OOP_Rotations_Degrees)
    c = ax.pcolor(x,y,energies,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Nonbonded Energies',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Nonbonded_Energies' % (names[0],names[1]))
    plt.close(fig)

    fig,ax = plt.subplots(1,1)
    c = ax.pcolor(x,y,reg_tracker,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Tracker',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Hydrogenated_Tracker' % (names[0],names[1]))
    plt.close(fig)

    fig,ax = plt.subplots(1,1)
    c = ax.pcolor(x,y,improper_tracker,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Tracker',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Hydrogenated_Improper_Tracker' % (names[0],names[1]))
    plt.close(fig)

for names,energies in zip(Dimer_Names,E_delocalization):
    energies = energies - np.amin(energies[0])
    fig,ax = plt.subplots(1,1)
    x,y = np.meshgrid(Dih_Rotations_Degrees,OOP_Rotations_Degrees)
    c = ax.pcolor(x,y,energies,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Delocalization Energies',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Delocalization_Energies' % (names[0],names[1]))
    plt.close(fig)

    Normalized_Energy_List = []

    fig,ax = plt.subplots(1,1)
    for energy_list in energies:
        norm_energies = energy_list - np.amin(energy_list)
        plt.scatter(np.linspace(0,350,Rotated_Shape[1]),energy_list)
        Normalized_Energy_List.append(norm_energies)

    plt.ylabel('Energy (kcal/mol)',size = 24)
    plt.xlabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    plt.ylim([-10,10])
    fig.savefig('%s_%s_Delocalization_Energies_Row_By_Row' % (names[0],names[1]))
    plt.close(fig)


    for i in range(len(energies)):
        fig,ax = plt.subplots(1,1)
        for energy_list in energies[:i]:
            plt.scatter(np.linspace(0,350,Rotated_Shape[1]),energy_list,alpha=0.2,marker = 's',c='k')

        plt.scatter(np.linspace(0,350,Rotated_Shape[1]),energies[i],marker = 's',c='k')
        plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
        plt.ylabel('Energy (kcal/mol)',size = 24)
        ax.tick_params(axis="x", labelsize=18)
        ax.tick_params(axis="y", labelsize=18)
        ax.tick_params(length=4,width=4)
        plt.ylim([-10,15])
        plt.tight_layout()
        fig.savefig('%s_%s_Delocalization_Energies_Row_By_Row_%d' % (names[0],names[1],i))
        plt.close(fig)
        os.system("scp %s_%s_Delocalization_Energies_Row_By_Row_%d.png ./Figures" % (names[0],names[1],i))
        os.system("rm -f %s_%s_Delocalization_Energies_Row_By_Row_%d.png" % (names[0],names[1],i))

    for i in range(len(energies[0])):
        fig,ax = plt.subplots(1,1)

        plt.scatter(np.linspace(0,350,Rotated_Shape[1]),energies[0],marker = 's',c='k')
        plt.scatter([np.linspace(0,350,Rotated_Shape[1])[i]],[energies[0][i]],marker = 's',c='k',edgecolors='r',linewidths = 2.25)
        plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
        plt.ylabel('Energy (kcal/mol)',size = 24)
        ax.tick_params(axis="x", labelsize=18)
        ax.tick_params(axis="y", labelsize=18)
        ax.tick_params(length=4,width=4)
        plt.ylim([-10,15])
        plt.tight_layout()
        fig.savefig('%s_%s_Delocalization_Energies_First_Row_%d' % (names[0],names[1],i))
        plt.close(fig)
        os.system("scp %s_%s_Delocalization_Energies_First_Row_%d.png ./Figures" % (names[0],names[1],i))
        os.system("rm -f %s_%s_Delocalization_Energies_First_Row_%d.png" % (names[0],names[1],i))

    fig,ax = plt.subplots(1,1)
    x,y = np.meshgrid(Dih_Rotations_Degrees,OOP_Rotations_Degrees)
    c = ax.pcolor(x,y,Normalized_Energy_List,cmap = 'seismic',vmin=0,vmax=10)
    ax.set_title('Delocalization Energies (Adjusted)',fontdict = {'fontsize':24})
    plt.xlabel('Dihedral Angle ($^\circ$)',size = 24)
    plt.ylabel('OOP Angle ($^\circ$)',size = 24)
    ax.tick_params(axis="x", labelsize=18)
    ax.tick_params(axis="y", labelsize=18)
    ax.tick_params(length=4,width=4)
    fig.savefig('%s_%s_Delocalization_Energies_Normalized' % (names[0],names[1]))
    plt.close(fig)
